# Übungsaufgaben DQL

Der folgende Block enthält Übungsaufgaben zu den DQL-Kapiteln.

## SELECT

Löse diese Aufgaben, nachdem du den SELECT-Block bearbeitet hast.

**1.1** Zeige alle verfügbaren (aktiven) Kunden an.

In [ ]:
USE WideWorldImporters
GO
SELECT * FROM sales.Customers

**1.2** Zeige die Vor- und Nachnamen aller Benutzer an

In [ ]:
SELECT FullName FROM application.People;

**1.3** Welche Liefermethoden kennt WWI?

In [ ]:
SELECT deliverymethodname FROM Application.DeliveryMethods

**1.4** Erstelle eine Abfrage auf die Tabelle _Sales.Customers,_ welche dir folgende Felder liefert: CustomerId, CustomerName, Address (Bestehend als konkatiniertes Feld aus den Feldern PostalAddressLine1, PostalAddressLine2 und PostalPostalCode).

In [ ]:
SELECT CustomerId, CustomerName, PostalAddressLine1 +' '+ PostalAddressLine2+' '+PostalPostalCode AS ADRESS FROM Sales.Customers

## WHERE-Bedingungen

Führe diese Aufgaben aus, nachdem du den Block "WHERE-Bedingungen" bearbeitet hast.

**2.1** Wie viele Mitarbeiter zählt WWI?

In [ ]:
SELECT COUNT(*) AS 'Anzahl Mitarbeiter' FROM [WideWorldImporters].[Application].[People]
WHERE IsEmployee = 1

**2.2** Wie viele Mitarbeiter sind Verkäufer?

In [ ]:
SELECT COUNT(*) FROM [WideWorldImporters].[Application].[People]
WHERE IsSalesperson = 1

**2.3**: Gebe alle Städte der Städte-Tabelle aus, welche mehr als 10'000 Einwohner haben.

In [ ]:
SELECT DISTINCT CityName, LatestRecordedPopulation FROM [WideWorldImporters].[Application].[Cities]
WHERE LatestRecordedPopulation >10000
ORDER BY LatestRecordedPopulation DESC

**2.4** Liste alle Städte auf, welche mit "Ab" beginnen.

In [ ]:
SELECT DISTINCT CITYNAME FROM APPLICATION.Cities
WHERE CITYNAME LIKE 'AB%'

## JOIN

Führe diese Aufgaben aus, nachdem du den JOIN-Block bearbeitet hast.

**3.1** Erstelle eine Abfrage, welche die Tabelle _Customers_ mit der Tabelle _Orders_ vereint. Es sollen nur Kunden ausgegeben werden, welche etwas bestellt haben. Die Abfrage soll mindestens die Felder _CustomerId_ und _OrderId_ enthalten.

In [ ]:
SELECT * FROM Sales.Customers c
JOIN Sales.orders o ON c.CustomerID=o.CustomerID


**3.2** Welche Kunden sind in der Datenbank erfasst, welche über keine Bestellungen verfügen? Liste explizit die Kunden auf, welche über keine Bestellung verfügen.

In [ ]:
SELECT CustomerID FROM Sales.customers
EXCEPT
SELECT CustomerID FROM Sales.orders


In [ ]:
SELECT c.customerid, o.customerid FROM Sales.customers c
LEFT JOIN Sales.orders o ON c.customerid = o.customerid
WHERE o.customerid IS NULL


**3.3** Gebe für jeden Kunden die dazugehörige Rechnungsadresse aus. (Tipp: Das Feld _BillToCustomerId_ zeigt dir, wer der Rechnungsempfänger ist).

In [ ]:
SELECT CustomerName, DeliveryAddressLine1+' '+DeliveryAddressLine2+' '+DeliveryPostalCode+' '
+PostalAddressLine1+' '+PostalAddressLine2 AS ADRESSE FROM Sales.customers
WHERE CustomerID IN (SELECT DISTINCT BillToCustomerID FROM Sales.customers)


## CASE

Führe diese Aufgaben aus, nachdem du den CASE-Block bearbeitet hast.

**4.1** Erstelle folgende Abfrage: Liste Personen auf, die entweder Mitarbeiter, Verkäufer oder Kunde sind. Das Resultat soll dabei folgende Felder enthalten: Vollständiger Name, Telefon-Nummer, Fax-Nummer, Rolle. In der Rolle sollen folgende Werte abgebildet werden: Mitarbeiter, wenn eine Peron ein Mitarbeiter ist, Verkäufer, wenn eine Person Verkäufer ist, Kunde, wenn die Person weder Verkäufer noch Mitarbeiter ist. Einträge ohne Telefon unx Fax-Nummer sollen herausgefiltert werden.

In [ ]:
SELECT FullName, PhoneNumber, FaxNumber,
CASE
WHEN IsSystemUser = 1 AND IsEmployee = 1 AND isSalesperson =1 THEN 'USER / Mitarbeiter / VERKÄUFER' 
WHEN IsSystemUser = 0 AND IsEmployee = 1 AND isSalesperson =1 THEN 'Mitarbeiter / VERKÄUFER'
WHEN IsSystemUser = 1 AND IsEmployee = 0 AND isSalesperson =1 THEN 'USER / VERKÄUFER'
WHEN IsSystemUser = 1 AND IsEmployee = 1 AND isSalesperson =0 THEN 'USER / Mitarbeiter'
WHEN IsSystemUser = 1 AND IsEmployee = 0 AND isSalesperson =0 THEN 'USER' 
WHEN IsSystemUser = 0 AND IsEmployee = 0 AND isSalesperson =1 THEN 'VERKÄUFER' 
WHEN IsSystemUser = 0 AND IsEmployee = 1 AND isSalesperson =0 THEN 'MITARBEITER' 

ELSE 'KUNDE '
END AS 'ROLLE'
FROM Application.People
WHERE PhoneNumber IS NOT NULL OR FaxNumber IS NOT NULL

## Gruppierungen

Führe diese Aufgaben aus, nachdem du den Gruppierungs-Block bearbeitet hast.

**5.1** Finde die zehn teuersten Bestellungen, welche WWI entgegennehmen durfte

In [ ]:
SELECT TOP (10) ol.OrderID, SUM(ol.Quantity*ol.UnitPrice) AS 'TOTAL' FROM Sales.OrderLines ol
GROUP BY ol.OrderID
ORDER BY TOTAL DESC

**5.2** Zu welchen Kunden gehören die zehn teuersten Bestellungen?

In [3]:
USE WideWorldImporters
GO

SELECT c.CustomerName FROM Sales.Customers c
Join Sales.Orders o ON c.CustomerID=o.CustomerID 
JOIN 
(SELECT TOP (10) ol.OrderID, SUM(ol.Quantity*ol.UnitPrice) AS 'TOTAL' FROM Sales.OrderLines ol
GROUP BY ol.OrderID
ORDER BY TOTAL DESC) u ON o.OrderID = u.OrderID 

Commands completed successfully.

(10 rows affected)

Total execution time: 00:00:00.208

CustomerName
"Tailspin Toys (Medicine Lodge, KS)"
"Tailspin Toys (Good Hart, MI)"
"Tailspin Toys (New Lexington, OH)"
"Wingtip Toys (Leathersville, GA)"
"Wingtip Toys (Taft Heights, CA)"
"Wingtip Toys (Nichols Hills, OK)"
"Wingtip Toys (Knights Landing, CA)"
"Wingtip Toys (Dunkerton, IA)"
Cong Hoa
Raj Verma


**5.3** Welches sind die zehn Umsatzstärksten Kunden von WWI?

In [6]:
USE WideWorldImporters
GO

SELECT TOP 10 c.CustomerName, SUM(ol.UnitPrice*ol.Quantity) AS GESAMMTUMSATZ FROM Sales.Customers c
JOIN Sales.Orders o ON c.CustomerID=o.CustomerID
JOIN Sales.OrderLines ol ON o.OrderID=ol.OrderID
GROUP BY c.CustomerName
ORDER BY GESAMMTUMSATZ DESC;

Commands completed successfully.

(10 rows affected)

Total execution time: 00:00:00.086

CustomerName,GESAMMTUMSATZ
"Tailspin Toys (Inguadona, MN)",384393.35
"Tailspin Toys (Minidoka, ID)",379660.70
Mauno Laurila,377189.80
"Wingtip Toys (Sarversville, PA)",372350.00
Ingrida Zeltina,368067.45
"Tailspin Toys (Long Meadow, MD)",367258.50
Nasrin Omidzadeh,366883.75
"Wingtip Toys (Cuyamungue, NM)",365915.45
"Wingtip Toys (San Jacinto, CA)",365330.95
"Wingtip Toys (Morrison Bluff, AR)",360652.80


**5.4** Wie viele Provinzen haben die einzelnen hinterlegten Lieferländer von WWI? Gebe dazu die Anzahl Provinzen gruppiert nach Land aus.

In [ ]:
SELECT ac.CountryName,COUNT(asp.StateProvinceID) AS Provinzanzahl  FROM Application.Countries ac
JOIN Application.StateProvinces asp ON ac.CountryID=asp.CountryID
GROUP BY ac.CountryName

**5.5** Wie setzt sich das Resultat aus 2.14.4 zusammen? Erläutere die Zusammenhänge.

**5.6** Finde alle Bestellungen, welche mehr oder genau 5 Bestellpositionen haben.

In [ ]:
SELECT OrderID, COUNT(OrderLineID)AS ANZAHL_LINES FROM Sales.OrderLines
GROUP BY OrderID
HAVING COUNT(OrderLineID)>=5

## UNION

Führe diese Aufgaben aus, nachdem du den UNION-Block bearbeitet hast.

**6.1** Welche Kundenkategorien kannte WWI über die gesamte Laufbahn? Liste alle namentlich auf.

In [ ]:
SELECT CustomerCategoryID,CustomerCategoryName FROM Sales.CustomerCategories

WHERE CustomerCategoryID IN(SELECT DISTINCT CustomerCategoryID FROM Sales.Customers)

## Spezialaufgaben

Führe die folgenden Aufgaben aus, nachdem du die anderen Blöcke fertig bearbeitet hast

**7.1** Erstelle eine Gesamtübersicht über die WWI für das Jahr 2013, die folgende Informationen enthält: Anzahl Kunden, Anzahl Bestellungen im Jahre 2013, Bestellvolumen Jan. - Mär. 2013, Bestellvolumen Mär. - Jun. 2013, Bestellvolumen Jul. - Sep. 2013, Bestellvolumen Okt - Dez. 2013.

In [ ]:
SELECT 
U1.[Anzahl Kunden],
U2.[Anzahl Bestellungen im Jahre 2013],
U3.[Bestellvolumen Jan. - Mär. 2013],
U4.[Bestellvolumen April. - Jun. 2013],
U5.[Bestellvolumen Jul. - Sep. 2013],
U6.[Bestellvolumen Okt - Dez. 2013]

FROM 

(SELECT COUNT(CustomerID) AS 'Anzahl Kunden' FROM SALES.Customers)U1 ,


(SELECT COUNT(OrderID) AS 'Anzahl Bestellungen im Jahre 2013' FROM SALES.Orders
WHERE YEAR(OrderDate)=2013) U2 ,

(SELECT COUNT(OrderID) AS 'Bestellvolumen Jan. - Mär. 2013' FROM SALES.Orders
WHERE OrderDate BETWEEN '2013.01.01' AND '2013.03.31')U3 ,

(SELECT COUNT(OrderID) AS 'Bestellvolumen April. - Jun. 2013' FROM SALES.Orders
WHERE OrderDate BETWEEN '2013.04.01' AND '2013.06.30') U4 ,

(SELECT COUNT(OrderID) AS 'Bestellvolumen Jul. - Sep. 2013' FROM SALES.Orders
WHERE OrderDate BETWEEN '2013.07.01' AND '2013.09.30') U5 ,

(SELECT COUNT(OrderID) AS 'Bestellvolumen Okt - Dez. 2013' FROM SALES.Orders
WHERE OrderDate BETWEEN '2013.10.01' AND '2013.12.31') U6


**7.2** Ermittle das gesamte Bestellvolumen für die Jahre 2013 bis 2016. Dabei soll für jedes Jahr eine eigene Spalte verwendet werden.

In [6]:
SELECT 
U13.UMSATZ_2013, U14.UMSATZ_2014,U15.UMSATZ_2015,U16.UMSATZ_2016
FROM

(SELECT 
SUM(Quantity*UnitPrice) AS UMSATZ_2013
 FROM Sales.OrderLines
WHERE OrderID IN
(SELECT OrderID FROM Sales.Orders
WHERE YEAR(OrderDate)= 2013))U13,

(SELECT 
SUM(Quantity*UnitPrice) AS UMSATZ_2014
 FROM Sales.OrderLines
WHERE OrderID IN
(SELECT OrderID FROM Sales.Orders
WHERE YEAR(OrderDate)= 2014))U14,

(SELECT 
SUM(Quantity*UnitPrice) AS UMSATZ_2015
 FROM Sales.OrderLines
WHERE OrderID IN
(SELECT OrderID FROM Sales.Orders
WHERE YEAR(OrderDate)= 2015))U15,

(SELECT 
SUM(Quantity*UnitPrice) AS UMSATZ_2016
 FROM Sales.OrderLines
WHERE OrderID IN
(SELECT OrderID FROM Sales.Orders
WHERE YEAR(OrderDate)= 2016))U16

(1 row affected)

Total execution time: 00:00:00.280

UMSATZ_2013,UMSATZ_2014,UMSATZ_2015,UMSATZ_2016
46928592.80,51492003.40,55817887.45,23395792.75
